# 🔗 LangChain链系统入门

## 🎯 学习目标

在本课程中，你将学习：
- Chain（链）的核心概念和原理
- 基础链类型和使用方法
- 自定义链的创建和配置
- 链的组合和嵌套技术
- 错误处理和调试技巧
- 实际应用场景和最佳实践

---

In [ ]:
# 初始化学习环境
import sys
import os
sys.path.append('../utils')

from progress_tracker import ProgressTracker
import warnings
warnings.filterwarnings('ignore')

# 初始化进度追踪器
tracker = ProgressTracker()
lesson_id = "01_chains_introduction"
tracker.start_lesson(lesson_id, "LangChain链系统入门")

print("🚀 欢迎来到LangChain链系统课程！")
print("📊 正在初始化学习环境...")

# 显示学习进度
tracker.display_progress_summary()

## 1. 🔍 什么是Chain（链）？

### 1.1 基本概念

Chain是LangChain的核心概念，它将多个组件（如提示词、LLM、输出解析器）连接成一个完整的处理流程。

In [ ]:
# 环境配置检查
try:
    from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
    from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
    from langchain_core.runnables import RunnablePassthrough, RunnableLambda
    from langchain_openai import ChatOpenAI
    from dotenv import load_dotenv
    import json
    
    # 加载环境变量
    load_dotenv()
    
    print("✅ LangChain链模块加载成功")
    
    # 检查API密钥
    if os.getenv('OPENAI_API_KEY'):
        print("✅ OpenAI API密钥已配置")
        has_api_key = True
        # 初始化LLM（使用较小的模型以节省成本）
        llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=500
        )
    else:
        print("⚠️ 未检测到OpenAI API密钥，将使用模拟模式")
        has_api_key = False
        
        # 创建模拟LLM类
        class MockLLM:
            def invoke(self, prompt):
                return f"[模拟响应] 基于提示词: {prompt[:50]}..."
            
            def __call__(self, prompt):
                return self.invoke(prompt)
        
        llm = MockLLM()
        
except ImportError as e:
    print(f"❌ 导入错误: {e}")
    print("💡 请运行: pip install langchain langchain-openai")
    has_api_key = False

### 1.2 Chain的组成部分

In [ ]:
print("🧩 Chain的核心组成部分：")
print("="*50)

components = {
    "📝 提示词模板 (PromptTemplate)": "定义输入格式和指令",
    "🤖 语言模型 (LLM)": "处理文本生成的核心引擎",
    "🔧 输出解析器 (OutputParser)": "处理和格式化LLM输出",
    "🔗 连接器 (Runnable)": "连接各个组件的管道",
    "📊 记忆系统 (Memory)": "存储对话历史和上下文",
    "🛠️ 工具集 (Tools)": "扩展链的功能和能力"
}

for component, description in components.items():
    print(f"{component}: {description}")

print("\n💡 链的工作流程：")
workflow = [
    "1. 接收用户输入",
    "2. 应用提示词模板格式化",
    "3. 发送给语言模型处理",
    "4. 解析和格式化输出",
    "5. 返回最终结果"
]

for step in workflow:
    print(f"   {step}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "chain_concepts")

## 2. 🔧 基础链类型

### 2.1 简单LLM链

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 创建简单的提示词模板
prompt_template = PromptTemplate(
    input_variables=["topic", "audience"],
    template="""
请为{audience}写一篇关于{topic}的简短介绍文章。

要求：
- 内容准确且易懂
- 长度控制在200字以内
- 语言风格适合目标受众

文章：
"""
)

# 创建输出解析器
output_parser = StrOutputParser()

# 构建链（使用LCEL - LangChain Expression Language）
simple_chain = prompt_template | llm | output_parser

print("🔗 简单LLM链示例：")
print("链结构: PromptTemplate | LLM | OutputParser")

# 测试链
try:
    if has_api_key:
        result = simple_chain.invoke({
            "topic": "机器学习",
            "audience": "高中学生"
        })
        print(f"\n✅ 链执行结果：\n{result}")
    else:
        # 模拟执行
        formatted_prompt = prompt_template.format(
            topic="机器学习",
            audience="高中学生"
        )
        print(f"\n📝 格式化的提示词：\n{formatted_prompt}")
        print("\n🤖 模拟输出: [这里会是LLM生成的关于机器学习的高中生介绍文章]")
        
except Exception as e:
    print(f"❌ 链执行失败: {e}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "simple_llm_chain")

### 2.2 顺序链 (Sequential Chain)

In [ ]:
from langchain_core.runnables import RunnableLambda

# 第一步：生成文章大纲
outline_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""
请为主题"{topic}"创建一个文章大纲。

要求：
- 包含3-5个主要部分
- 每个部分有简短说明
- 逻辑清晰，结构合理

大纲：
"""
)

# 第二步：基于大纲写文章
article_prompt = PromptTemplate(
    input_variables=["topic", "outline"],
    template="""
主题：{topic}
大纲：{outline}

请根据上述大纲写一篇完整的文章。

要求：
- 严格按照大纲结构
- 内容充实，逻辑清晰
- 语言流畅，表达准确

文章：
"""
)

# 第三步：生成文章摘要
summary_prompt = PromptTemplate(
    input_variables=["article"],
    template="""
请为以下文章生成一个简洁的摘要：

{article}

摘要要求：
- 50字以内
- 突出核心要点
- 语言简洁明了

摘要：
"""
)

# 构建顺序链
def create_sequential_chain():
    """创建顺序链：大纲 -> 文章 -> 摘要"""
    
    # 步骤1：生成大纲
    outline_chain = outline_prompt | llm | StrOutputParser()
    
    # 步骤2：基于大纲写文章
    def create_article_input(inputs):
        """组合大纲和主题作为文章生成的输入"""
        return {
            "topic": inputs["topic"],
            "outline": inputs["outline"]
        }
    
    article_chain = article_prompt | llm | StrOutputParser()
    
    # 步骤3：生成摘要
    summary_chain = summary_prompt | llm | StrOutputParser()
    
    # 组合完整的顺序链
    full_chain = {
        "topic": RunnablePassthrough(),
        "outline": outline_chain
    } | RunnableLambda(create_article_input) | {
        "article": article_chain,
        "topic": RunnablePassthrough()
    } | {
        "summary": summary_chain,
        "article": lambda x: x["article"],
        "topic": lambda x: x["topic"]
    }
    
    return full_chain

print("🔗 顺序链示例：大纲 → 文章 → 摘要")
print("链结构: [主题] → [大纲生成] → [文章写作] → [摘要生成]")

# 测试顺序链
if has_api_key:
    try:
        sequential_chain = create_sequential_chain()
        result = sequential_chain.invoke({"topic": "人工智能的发展历程"})
        
        print("\n✅ 顺序链执行结果：")
        print(f"主题: {result['topic']}")
        print(f"\n文章摘要: {result['summary']}")
        print(f"\n完整文章: {result['article'][:200]}...")
        
    except Exception as e:
        print(f"❌ 顺序链执行失败: {e}")
else:
    # 模拟执行流程
    print("\n🔄 模拟顺序链执行流程：")
    steps = [
        "1. 输入主题: '人工智能的发展历程'",
        "2. 生成大纲: [AI起源 → 发展阶段 → 现状 → 未来]",
        "3. 基于大纲写文章: [详细的AI发展历程文章]",
        "4. 生成摘要: [50字内的核心要点摘要]"
    ]
    
    for step in steps:
        print(f"   {step}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "sequential_chain")

### 2.3 并行链 (Parallel Chain)

In [ ]:
from langchain_core.runnables import RunnableParallel

# 定义多个并行处理的子链

# 子链1：技术分析
technical_prompt = PromptTemplate(
    input_variables=["product"],
    template="""
从技术角度分析产品"{product}"：

请分析：
- 核心技术特点
- 技术优势
- 技术挑战
- 技术发展趋势

技术分析：
"""
)

# 子链2：市场分析
market_prompt = PromptTemplate(
    input_variables=["product"],
    template="""
从市场角度分析产品"{product}"：

请分析：
- 目标市场
- 竞争环境
- 市场机会
- 风险因素

市场分析：
"""
)

# 子链3：用户体验分析
ux_prompt = PromptTemplate(
    input_variables=["product"],
    template="""
从用户体验角度分析产品"{product}"：

请分析：
- 用户需求匹配度
- 使用便利性
- 用户满意度
- 改进建议

用户体验分析：
"""
)

# 创建并行链
parallel_chain = RunnableParallel({
    "technical_analysis": technical_prompt | llm | StrOutputParser(),
    "market_analysis": market_prompt | llm | StrOutputParser(),
    "ux_analysis": ux_prompt | llm | StrOutputParser()
})

# 综合分析链
synthesis_prompt = PromptTemplate(
    input_variables=["technical", "market", "ux", "product"],
    template="""
产品：{product}

技术分析：{technical}

市场分析：{market}

用户体验分析：{ux}

请基于以上三个角度的分析，提供一个综合的产品评估和建议：

综合评估：
"""
)

# 完整的并行-综合链
def create_comprehensive_analysis_chain():
    """创建综合分析链：并行分析 + 综合评估"""
    
    def prepare_synthesis_input(parallel_results):
        """准备综合分析的输入"""
        return {
            "product": parallel_results["product"],
            "technical": parallel_results["technical_analysis"],
            "market": parallel_results["market_analysis"],
            "ux": parallel_results["ux_analysis"]
        }
    
    # 组合链：并行分析 -> 综合评估
    full_chain = {
        "product": RunnablePassthrough(),
        **parallel_chain.map()
    } | RunnableLambda(prepare_synthesis_input) | synthesis_prompt | llm | StrOutputParser()
    
    return full_chain

print("🔗 并行链示例：多角度产品分析")
print("链结构: [产品] → [技术分析 || 市场分析 || 用户体验分析] → [综合评估]")

# 测试并行链
if has_api_key:
    try:
        # 仅测试并行分析部分（避免复杂的链组合）
        test_input = {"product": "ChatGPT"}
        parallel_results = parallel_chain.invoke(test_input)
        
        print("\n✅ 并行链执行结果：")
        print(f"\n🔧 技术分析：\n{parallel_results['technical_analysis'][:150]}...")
        print(f"\n📈 市场分析：\n{parallel_results['market_analysis'][:150]}...")
        print(f"\n👥 用户体验分析：\n{parallel_results['ux_analysis'][:150]}...")
        
    except Exception as e:
        print(f"❌ 并行链执行失败: {e}")
else:
    # 模拟执行流程
    print("\n🔄 模拟并行链执行流程：")
    print("输入: 'ChatGPT'")
    print("\n同时执行三个分析：")
    print("   🔧 技术分析: [GPT架构、训练方法、技术创新...]")
    print("   📈 市场分析: [AI市场、竞争对手、商业模式...]")
    print("   👥 用户体验: [易用性、响应速度、满意度...]")
    print("\n最后综合: [基于三个分析的综合评估和建议]")

# 记录完成状态
tracker.complete_exercise(lesson_id, "parallel_chain")

## 3. 🛠️ 自定义链的创建

### 3.1 创建自定义数据处理链

In [ ]:
from langchain_core.runnables import RunnableLambda
from typing import Dict, Any, List
import re
import json

class CustomDataProcessingChain:
    """自定义数据处理链"""
    
    def __init__(self, llm):
        self.llm = llm
        self.setup_chain()
    
    def data_cleaner(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """数据清理步骤"""
        text = data.get("text", "")
        
        # 基础清理
        cleaned_text = re.sub(r'\s+', ' ', text)  # 标准化空白字符
        cleaned_text = re.sub(r'[^\w\s\u4e00-\u9fff.,!?]', '', cleaned_text)  # 保留基本字符
        cleaned_text = cleaned_text.strip()
        
        return {
            "original_text": text,
            "cleaned_text": cleaned_text,
            "char_count": len(cleaned_text),
            "word_count": len(cleaned_text.split())
        }
    
    def sentiment_analyzer(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """情感分析步骤"""
        text = data["cleaned_text"]
        
        sentiment_prompt = PromptTemplate(
            input_variables=["text"],
            template="""
请分析以下文本的情感倾向：

文本：{text}

请输出JSON格式的分析结果：
{{
    "sentiment": "positive/negative/neutral",
    "confidence": 0.0-1.0,
    "key_emotions": ["emotion1", "emotion2"],
    "reasoning": "分析理由"
}}

分析结果：
"""
        )
        
        if hasattr(self.llm, 'invoke'):
            # 真实LLM调用
            try:
                chain = sentiment_prompt | self.llm | StrOutputParser()
                result = chain.invoke({"text": text})
                
                # 尝试解析JSON
                try:
                    sentiment_data = json.loads(result)
                except json.JSONDecodeError:
                    # 如果解析失败，使用默认值
                    sentiment_data = {
                        "sentiment": "neutral",
                        "confidence": 0.5,
                        "key_emotions": ["neutral"],
                        "reasoning": "解析失败，使用默认值"
                    }
            except:
                sentiment_data = {
                    "sentiment": "neutral",
                    "confidence": 0.5,
                    "key_emotions": ["neutral"],
                    "reasoning": "LLM调用失败"
                }
        else:
            # 模拟情感分析
            positive_words = ['好', '棒', '优秀', '喜欢', '满意', 'good', 'great', 'excellent']
            negative_words = ['坏', '差', '糟糕', '讨厌', '失望', 'bad', 'terrible', 'awful']
            
            positive_count = sum(1 for word in positive_words if word in text.lower())
            negative_count = sum(1 for word in negative_words if word in text.lower())
            
            if positive_count > negative_count:
                sentiment = "positive"
                confidence = min(0.9, 0.6 + positive_count * 0.1)
            elif negative_count > positive_count:
                sentiment = "negative"
                confidence = min(0.9, 0.6 + negative_count * 0.1)
            else:
                sentiment = "neutral"
                confidence = 0.7
            
            sentiment_data = {
                "sentiment": sentiment,
                "confidence": confidence,
                "key_emotions": [sentiment],
                "reasoning": f"基于关键词分析：积极词{positive_count}个，消极词{negative_count}个"
            }
        
        # 合并数据
        data.update(sentiment_data)
        return data
    
    def summary_generator(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """摘要生成步骤"""
        text = data["cleaned_text"]
        sentiment = data["sentiment"]
        
        summary_prompt = PromptTemplate(
            input_variables=["text", "sentiment"],
            template="""
请为以下文本生成摘要：

文本：{text}
情感倾向：{sentiment}

请生成一个30字以内的摘要，突出核心内容和情感特征：

摘要：
"""
        )
        
        if hasattr(self.llm, 'invoke'):
            try:
                chain = summary_prompt | self.llm | StrOutputParser()
                summary = chain.invoke({"text": text, "sentiment": sentiment})
            except:
                summary = f"[{sentiment}情感] {text[:20]}..."
        else:
            # 简单摘要生成
            words = text.split()[:10]  # 取前10个词
            summary = f"[{sentiment}情感] {' '.join(words)}..."
        
        data["summary"] = summary
        return data
    
    def setup_chain(self):
        """设置处理链"""
        self.chain = (
            RunnableLambda(self.data_cleaner) |
            RunnableLambda(self.sentiment_analyzer) |
            RunnableLambda(self.summary_generator)
        )
    
    def process(self, text: str) -> Dict[str, Any]:
        """处理文本"""
        return self.chain.invoke({"text": text})

# 创建自定义处理链实例
custom_chain = CustomDataProcessingChain(llm)

print("🛠️ 自定义数据处理链")
print("处理流程: 数据清理 → 情感分析 → 摘要生成")

# 测试自定义链
test_texts = [
    "这个产品真的太棒了！我非常满意，推荐给所有人。界面设计很好，功能也很强大。",
    "使用体验很差，bug太多了，完全不推荐。客服态度也不好，很失望。",
    "产品还可以，有优点也有缺点。价格合理，但功能需要改进。"
]

for i, text in enumerate(test_texts, 1):
    print(f"\n📝 测试文本{i}：{text[:30]}...")
    
    try:
        result = custom_chain.process(text)
        
        print(f"   情感：{result['sentiment']} (置信度: {result['confidence']:.2f})")
        print(f"   摘要：{result['summary']}")
        print(f"   字数：{result['char_count']} | 词数：{result['word_count']}")
        
    except Exception as e:
        print(f"   ❌ 处理失败: {e}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "custom_chain")

### 3.2 链的错误处理和调试

In [ ]:
from langchain_core.runnables import RunnableLambda
import logging
import time
from typing import Union

class RobustChain:
    """具有错误处理和调试功能的链"""
    
    def __init__(self, llm, debug=True):
        self.llm = llm
        self.debug = debug
        self.execution_log = []
        
        # 配置日志
        if debug:
            logging.basicConfig(level=logging.INFO)
            self.logger = logging.getLogger(__name__)
    
    def log_step(self, step_name: str, input_data: Any, output_data: Any = None, error: Exception = None):
        """记录执行步骤"""
        log_entry = {
            "timestamp": time.time(),
            "step": step_name,
            "input": str(input_data)[:100] + "..." if len(str(input_data)) > 100 else str(input_data),
            "output": str(output_data)[:100] + "..." if output_data and len(str(output_data)) > 100 else str(output_data),
            "error": str(error) if error else None,
            "status": "error" if error else "success"
        }
        
        self.execution_log.append(log_entry)
        
        if self.debug:
            if error:
                self.logger.error(f"❌ {step_name}: {error}")
            else:
                self.logger.info(f"✅ {step_name}: 成功")
    
    def safe_step(self, step_name: str, func, input_data, fallback_value=None):
        """安全执行步骤，包含错误处理"""
        try:
            start_time = time.time()
            result = func(input_data)
            execution_time = time.time() - start_time
            
            self.log_step(step_name, input_data, result)
            
            if self.debug:
                print(f"⏱️ {step_name} 执行时间: {execution_time:.2f}秒")
            
            return result
            
        except Exception as e:
            self.log_step(step_name, input_data, error=e)
            
            if fallback_value is not None:
                print(f"⚠️ {step_name} 失败，使用备用值: {fallback_value}")
                return fallback_value
            else:
                print(f"❌ {step_name} 失败: {e}")
                raise e
    
    def input_validator(self, data):
        """输入验证步骤"""
        if not isinstance(data, dict):
            raise ValueError("输入必须是字典格式")
        
        if "text" not in data:
            raise ValueError("输入必须包含'text'字段")
        
        if not data["text"].strip():
            raise ValueError("文本内容不能为空")
        
        if len(data["text"]) > 1000:
            print("⚠️ 文本长度超过1000字符，将被截断")
            data["text"] = data["text"][:1000]
        
        return data
    
    def text_processor(self, data):
        """文本处理步骤"""
        text = data["text"]
        
        # 基础处理
        processed_text = text.strip().replace("\n", " ")
        
        # 模拟可能的处理错误
        if "ERROR" in text.upper():
            raise RuntimeError("检测到错误标记，处理中断")
        
        return {
            **data,
            "processed_text": processed_text,
            "processing_timestamp": time.time()
        }
    
    def llm_caller(self, data):
        """LLM调用步骤"""
        text = data["processed_text"]
        
        prompt = PromptTemplate(
            input_variables=["text"],
            template="请对以下文本进行分析和总结：\n\n{text}\n\n分析结果："
        )
        
        if hasattr(self.llm, 'invoke'):
            chain = prompt | self.llm | StrOutputParser()
            result = chain.invoke({"text": text})
        else:
            # 模拟LLM输出
            result = f"[模拟分析] 文本长度: {len(text)}字符，内容摘要: {text[:50]}..."
        
        return {
            **data,
            "llm_result": result,
            "llm_timestamp": time.time()
        }
    
    def output_formatter(self, data):
        """输出格式化步骤"""
        return {
            "original_text": data["text"],
            "processed_text": data["processed_text"],
            "analysis_result": data["llm_result"],
            "processing_info": {
                "text_length": len(data["text"]),
                "processing_time": data.get("llm_timestamp", 0) - data.get("processing_timestamp", 0)
            }
        }
    
    def process(self, input_data):
        """执行完整的处理流程"""
        print("🔄 开始执行链处理流程...")
        
        # 清空执行日志
        self.execution_log = []
        
        try:
            # 步骤1：输入验证
            data = self.safe_step(
                "输入验证", 
                self.input_validator, 
                input_data
            )
            
            # 步骤2：文本处理
            data = self.safe_step(
                "文本处理", 
                self.text_processor, 
                data,
                fallback_value={**data, "processed_text": data["text"], "processing_timestamp": time.time()}
            )
            
            # 步骤3：LLM调用
            data = self.safe_step(
                "LLM分析", 
                self.llm_caller, 
                data,
                fallback_value={**data, "llm_result": "[分析失败] 使用备用结果", "llm_timestamp": time.time()}
            )
            
            # 步骤4：输出格式化
            result = self.safe_step(
                "输出格式化", 
                self.output_formatter, 
                data
            )
            
            print("✅ 链处理完成")
            return result
            
        except Exception as e:
            print(f"❌ 链处理失败: {e}")
            return None
    
    def get_execution_log(self):
        """获取执行日志"""
        return self.execution_log
    
    def print_execution_summary(self):
        """打印执行摘要"""
        print("\n📊 执行摘要：")
        print("="*40)
        
        for i, log in enumerate(self.execution_log, 1):
            status_icon = "✅" if log["status"] == "success" else "❌"
            print(f"{i}. {status_icon} {log['step']}")
            
            if log["error"]:
                print(f"   错误: {log['error']}")

# 创建强健链实例
robust_chain = RobustChain(llm, debug=True)

print("🛡️ 强健链测试：错误处理和调试")

# 测试用例
test_cases = [
    {"text": "这是一个正常的测试文本，应该可以成功处理。"},
    {"text": ""},  # 空文本测试
    {"text": "这个文本包含ERROR标记，会触发处理错误。"},  # 错误触发测试
    {"invalid_key": "无效的输入格式"},  # 无效输入测试
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n🧪 测试用例{i}：")
    result = robust_chain.process(test_case)
    
    if result:
        print(f"📤 输出: {result['analysis_result'][:100]}...")
    
    # 打印执行摘要
    robust_chain.print_execution_summary()

# 记录完成状态
tracker.complete_exercise(lesson_id, "error_handling")

## 4. 🎯 实战练习

### 练习1：构建智能客服链

In [ ]:
print("🎯 练习1：智能客服链")
print("="*50)

# 任务：创建一个智能客服处理链，包含：
# 1. 意图识别
# 2. 情绪分析
# 3. 知识库查询
# 4. 响应生成

class CustomerServiceChain:
    """智能客服处理链"""
    
    def __init__(self, llm):
        self.llm = llm
        # 简单的知识库
        self.knowledge_base = {
            "订单查询": "您可以在'我的订单'页面查看订单状态，或提供订单号让我帮您查询。",
            "退换货": "商品在7天内可以申请退换货，请确保商品未拆封。详细流程请查看退换货政策。",
            "支付问题": "我们支持支付宝、微信支付、银行卡等多种支付方式。如遇支付问题，请联系支付平台客服。",
            "配送查询": "订单发货后，您会收到物流信息。一般3-5个工作日送达，偏远地区可能需要7-10天。",
            "产品咨询": "请告诉我您想了解的具体产品信息，我会为您详细介绍产品特性和优势。",
            "投诉建议": "非常感谢您的反馈，我们会认真对待每一个建议，并持续改进我们的服务。"
        }
    
    def intent_recognition(self, user_input: str) -> str:
        """意图识别"""
        intent_prompt = PromptTemplate(
            input_variables=["user_input", "intents"],
            template="""
请识别用户输入的意图类别：

用户输入：{user_input}

可能的意图类别：{intents}

请从上述类别中选择最匹配的一个，只输出类别名称：
"""
        )
        
        intents = list(self.knowledge_base.keys())
        
        if hasattr(self.llm, 'invoke'):
            try:
                chain = intent_prompt | self.llm | StrOutputParser()
                result = chain.invoke({
                    "user_input": user_input,
                    "intents": ", ".join(intents)
                })
                
                # 简单匹配验证
                for intent in intents:
                    if intent in result:
                        return intent
                
                return "产品咨询"  # 默认意图
                
            except:
                pass
        
        # 简单关键词匹配作为备用
        user_lower = user_input.lower()
        
        if any(word in user_lower for word in ['订单', '查询', '单号']):
            return "订单查询"
        elif any(word in user_lower for word in ['退货', '换货', '退款']):
            return "退换货"
        elif any(word in user_lower for word in ['支付', '付款', '扣费']):
            return "支付问题"
        elif any(word in user_lower for word in ['物流', '配送', '快递']):
            return "配送查询"
        elif any(word in user_lower for word in ['投诉', '建议', '意见']):
            return "投诉建议"
        else:
            return "产品咨询"
    
    def emotion_analysis(self, user_input: str) -> dict:
        """情绪分析"""
        # 简单的情绪词典
        positive_words = ['满意', '喜欢', '好', '棒', '赞', '谢谢']
        negative_words = ['不满', '失望', '差', '烂', '糟糕', '愤怒', '投诉']
        urgent_words = ['急', '紧急', '立即', '马上', '尽快']
        
        user_lower = user_input.lower()
        
        emotion_score = 0
        urgency = False
        
        for word in positive_words:
            if word in user_lower:
                emotion_score += 1
        
        for word in negative_words:
            if word in user_lower:
                emotion_score -= 1
        
        for word in urgent_words:
            if word in user_lower:
                urgency = True
        
        if emotion_score > 0:
            emotion = "positive"
        elif emotion_score < 0:
            emotion = "negative"
        else:
            emotion = "neutral"
        
        return {
            "emotion": emotion,
            "urgency": urgency,
            "score": emotion_score
        }
    
    def generate_response(self, user_input: str, intent: str, emotion_info: dict) -> str:
        """生成响应"""
        # 从知识库获取基础回答
        base_answer = self.knowledge_base.get(intent, "感谢您的咨询，我会为您转接专业客服。")
        
        # 根据情绪调整语调
        if emotion_info["emotion"] == "negative":
            prefix = "我理解您的困扰，非常抱歉给您带来不便。"
        elif emotion_info["emotion"] == "positive":
            prefix = "感谢您的信任和支持！"
        else:
            prefix = "感谢您的咨询，"
        
        # 紧急情况处理
        if emotion_info["urgency"]:
            urgency_note = "\n\n⚡ 我注意到您的需求比较紧急，我会优先为您处理。"
        else:
            urgency_note = ""
        
        return f"{prefix}{base_answer}{urgency_note}"
    
    def process_customer_query(self, user_input: str) -> dict:
        """处理客户查询"""
        print(f"🤖 处理客户查询: {user_input}")
        
        # 步骤1：意图识别
        intent = self.intent_recognition(user_input)
        print(f"   🎯 识别意图: {intent}")
        
        # 步骤2：情绪分析
        emotion_info = self.emotion_analysis(user_input)
        print(f"   😊 情绪分析: {emotion_info['emotion']} (紧急: {emotion_info['urgency']})")
        
        # 步骤3：生成响应
        response = self.generate_response(user_input, intent, emotion_info)
        print(f"   💬 生成响应: {response[:50]}...")
        
        return {
            "user_input": user_input,
            "intent": intent,
            "emotion": emotion_info,
            "response": response
        }

# 创建客服链
customer_service = CustomerServiceChain(llm)

# 测试客服场景
test_queries = [
    "我的订单什么时候能发货？我比较急用",
    "这个产品质量太差了，我要退货！",
    "你们的服务很不错，想咨询一下新产品的功能",
    "支付的时候出现错误，扣了钱但是订单没成功",
    "物流信息三天没更新了，包裹到哪里了？"
]

print("\n🧪 客服链测试：")
for query in test_queries:
    print("\n" + "="*60)
    result = customer_service.process_customer_query(query)
    print(f"\n📝 最终响应：\n{result['response']}")

print("\n✅ 练习1完成！你已经学会了如何构建智能客服处理链。")

# 记录练习完成
tracker.complete_exercise(lesson_id, "exercise_customer_service")

### 练习2：多步骤数据分析链

In [ ]:
print("🎯 练习2：多步骤数据分析链")
print("="*50)

# 任务：创建一个数据分析链，能够：
# 1. 数据验证和清理
# 2. 统计分析
# 3. 趋势分析
# 4. 生成报告

import json
from typing import List, Dict

class DataAnalysisChain:
    """数据分析链"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def validate_and_clean_data(self, raw_data: List[Dict]) -> Dict:
        """数据验证和清理"""
        print("🔍 步骤1：数据验证和清理")
        
        valid_records = []
        errors = []
        
        for i, record in enumerate(raw_data):
            try:
                # 验证必需字段
                if not all(key in record for key in ['date', 'value']):
                    errors.append(f"记录{i}: 缺少必需字段")
                    continue
                
                # 数据类型转换
                cleaned_record = {
                    'date': record['date'],
                    'value': float(record['value']),
                    'category': record.get('category', 'unknown')
                }
                
                # 数据范围验证
                if cleaned_record['value'] < 0:
                    errors.append(f"记录{i}: 值不能为负数")
                    continue
                
                valid_records.append(cleaned_record)
                
            except Exception as e:
                errors.append(f"记录{i}: {str(e)}")
        
        print(f"   ✅ 有效记录: {len(valid_records)}")
        print(f"   ❌ 错误记录: {len(errors)}")
        
        return {
            'valid_data': valid_records,
            'errors': errors,
            'total_records': len(raw_data),
            'valid_rate': len(valid_records) / len(raw_data) if raw_data else 0
        }
    
    def statistical_analysis(self, cleaned_data: Dict) -> Dict:
        """统计分析"""
        print("📊 步骤2：统计分析")
        
        valid_data = cleaned_data['valid_data']
        
        if not valid_data:
            return {'error': '没有有效数据进行分析'}
        
        values = [record['value'] for record in valid_data]
        categories = [record['category'] for record in valid_data]
        
        # 基础统计
        stats = {
            'count': len(values),
            'sum': sum(values),
            'mean': sum(values) / len(values),
            'min': min(values),
            'max': max(values),
            'range': max(values) - min(values)
        }
        
        # 分类统计
        category_stats = {}
        for category in set(categories):
            category_values = [v for i, v in enumerate(values) if categories[i] == category]
            category_stats[category] = {
                'count': len(category_values),
                'sum': sum(category_values),
                'mean': sum(category_values) / len(category_values)
            }
        
        print(f"   📈 平均值: {stats['mean']:.2f}")
        print(f"   📉 范围: {stats['min']:.2f} - {stats['max']:.2f}")
        print(f"   📊 类别数: {len(category_stats)}")
        
        return {
            'basic_stats': stats,
            'category_stats': category_stats,
            'data_quality': {
                'valid_rate': cleaned_data['valid_rate'],
                'error_count': len(cleaned_data['errors'])
            }
        }
    
    def trend_analysis(self, data: List[Dict], stats: Dict) -> Dict:
        """趋势分析"""
        print("📈 步骤3：趋势分析")
        
        if len(data) < 2:
            return {'trend': 'insufficient_data'}
        
        # 简单趋势分析（基于时间序列）
        sorted_data = sorted(data, key=lambda x: x['date'])
        
        first_half = sorted_data[:len(sorted_data)//2]
        second_half = sorted_data[len(sorted_data)//2:]
        
        first_avg = sum(r['value'] for r in first_half) / len(first_half)
        second_avg = sum(r['value'] for r in second_half) / len(second_half)
        
        change_rate = (second_avg - first_avg) / first_avg * 100 if first_avg != 0 else 0
        
        if change_rate > 10:
            trend = 'strong_upward'
        elif change_rate > 5:
            trend = 'moderate_upward'
        elif change_rate > -5:
            trend = 'stable'
        elif change_rate > -10:
            trend = 'moderate_downward'
        else:
            trend = 'strong_downward'
        
        print(f"   🎯 趋势: {trend}")
        print(f"   📊 变化率: {change_rate:.2f}%")
        
        return {
            'trend': trend,
            'change_rate': change_rate,
            'first_period_avg': first_avg,
            'second_period_avg': second_avg
        }
    
    def generate_report(self, stats: Dict, trend: Dict, raw_info: Dict) -> str:
        """生成分析报告"""
        print("📝 步骤4：生成报告")
        
        report_prompt = PromptTemplate(
            input_variables=["stats", "trend", "data_quality"],
            template="""
请基于以下数据分析结果生成一份专业的分析报告：

统计数据：{stats}
趋势分析：{trend}
数据质量：{data_quality}

请生成包含以下部分的报告：
1. 数据概况
2. 关键发现
3. 趋势洞察
4. 建议和结论

报告：
"""
        )
        
        if hasattr(self.llm, 'invoke'):
            try:
                chain = report_prompt | self.llm | StrOutputParser()
                report = chain.invoke({
                    "stats": json.dumps(stats, ensure_ascii=False, indent=2),
                    "trend": json.dumps(trend, ensure_ascii=False, indent=2),
                    "data_quality": json.dumps(raw_info['data_quality'], ensure_ascii=False, indent=2)
                })
                return report
            except:
                pass
        
        # 生成简单报告
        basic_stats = stats['basic_stats']
        trend_desc = {
            'strong_upward': '强烈上升',
            'moderate_upward': '温和上升',
            'stable': '保持稳定',
            'moderate_downward': '温和下降',
            'strong_downward': '急剧下降'
        }.get(trend['trend'], '趋势不明')
        
        return f"""
📊 数据分析报告

1. 数据概况
   - 总记录数: {raw_info['total_records']}
   - 有效记录: {basic_stats['count']}
   - 数据质量: {raw_info['data_quality']['valid_rate']:.1%}

2. 关键发现
   - 平均值: {basic_stats['mean']:.2f}
   - 数值范围: {basic_stats['min']:.2f} - {basic_stats['max']:.2f}
   - 数据总和: {basic_stats['sum']:.2f}

3. 趋势洞察
   - 整体趋势: {trend_desc}
   - 变化率: {trend['change_rate']:.2f}%
   - 前期平均: {trend['first_period_avg']:.2f}
   - 后期平均: {trend['second_period_avg']:.2f}

4. 建议和结论
   - 数据质量良好，可信度高
   - 趋势变化明显，需要关注后续发展
   - 建议持续监控数据变化
"""
    
    def analyze(self, raw_data: List[Dict]) -> Dict:
        """执行完整的数据分析流程"""
        print("🔄 开始数据分析流程...")
        
        # 步骤1：数据清理
        cleaned_result = self.validate_and_clean_data(raw_data)
        
        # 步骤2：统计分析
        stats_result = self.statistical_analysis(cleaned_result)
        
        if 'error' in stats_result:
            return {'error': stats_result['error']}
        
        # 步骤3：趋势分析
        trend_result = self.trend_analysis(cleaned_result['valid_data'], stats_result)
        
        # 步骤4：生成报告
        report = self.generate_report(stats_result, trend_result, cleaned_result)
        
        print("✅ 数据分析完成")
        
        return {
            'statistics': stats_result,
            'trend': trend_result,
            'report': report,
            'data_info': cleaned_result
        }

# 创建数据分析链
data_analyzer = DataAnalysisChain(llm)

# 准备测试数据
test_data = [
    {'date': '2024-01-01', 'value': 100, 'category': 'A'},
    {'date': '2024-01-02', 'value': 110, 'category': 'A'},
    {'date': '2024-01-03', 'value': 95, 'category': 'B'},
    {'date': '2024-01-04', 'value': 120, 'category': 'A'},
    {'date': '2024-01-05', 'value': -10, 'category': 'C'},  # 无效数据
    {'date': '2024-01-06', 'value': 130, 'category': 'B'},
    {'date': '2024-01-07', 'value': 'invalid', 'category': 'A'},  # 无效数据
    {'date': '2024-01-08', 'value': 140, 'category': 'A'},
    {'date': '2024-01-09', 'value': 150, 'category': 'B'},
    {'value': 160, 'category': 'A'},  # 缺少日期
]

print("\n🧪 数据分析链测试：")
print("="*60)

result = data_analyzer.analyze(test_data)

if 'error' not in result:
    print("\n📋 分析报告：")
    print(result['report'])
else:
    print(f"❌ 分析失败: {result['error']}")

print("\n✅ 练习2完成！你已经学会了如何构建多步骤数据分析链。")

# 记录练习完成
tracker.complete_exercise(lesson_id, "exercise_data_analysis")

## 5. 📊 学习总结与进度回顾

In [ ]:
# 完成课程并生成学习报告
completion_time = tracker.complete_lesson(lesson_id)

print("🎓 LangChain链系统入门 - 课程完成！")
print("="*60)

print("\n📚 本课程核心知识点：")
key_concepts = [
    "Chain的基本概念和组成",
    "LCEL (LangChain Expression Language)语法",
    "简单LLM链的构建",
    "顺序链的设计和实现",
    "并行链的创建和优化",
    "自定义链的开发技巧",
    "错误处理和调试机制",
    "链的性能监控",
    "智能客服系统设计",
    "数据分析流程自动化"
]

for i, concept in enumerate(key_concepts, 1):
    print(f"   {i:2d}. {concept}")

print("\n🛠️ 实践技能收获：")
skills = [
    "掌握链的基础构建方法",
    "能够设计复杂的多步骤处理流程",
    "实现并行处理和结果合并",
    "构建强健的错误处理机制",
    "开发实际业务应用系统"
]

for i, skill in enumerate(skills, 1):
    print(f"   ⚡ {skill}")

print("\n🎯 最佳实践要点：")
best_practices = [
    "合理分解复杂任务为简单步骤",
    "为每个步骤添加错误处理",
    "使用并行处理提高效率",
    "实施日志记录和调试机制",
    "考虑性能优化和成本控制"
]

for practice in best_practices:
    print(f"   💡 {practice}")

print("\n🎯 下一步学习建议：")
next_steps = [
    "学习Agent智能代理系统（02_agents_basics.ipynb）",
    "探索记忆系统和上下文管理",
    "深入Chain的高级特性",
    "实践更复杂的业务场景"
]

for i, step in enumerate(next_steps, 1):
    print(f"   📈 {step}")

# 显示整体学习进度
print("\n" + "="*60)
tracker.display_progress_summary()

print("\n🎉 恭喜完成LangChain链系统入门课程！")
print("💪 你已经掌握了构建复杂AI应用的核心技术！")
print("🚀 继续学习Agent和Memory，构建更智能的应用！")